## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [32]:
#configure directory to import sast libraries
import sys 
import os 
#add sast library path
file_path = os.path.expanduser("~/random_sast/sast")
sys.path.append(file_path)
#add cd_diagram library path
file_path = os.path.expanduser("~/random_sast/cd_diagram")
sys.path.append(file_path)

#sys.path

In [33]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
import os

### Reading Datasets

In [34]:
# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)
df_other_methods=df_other_methods[['filename','folds:','0']]
df_other_methods['method']=df_other_methods['filename'].str.split('_').str[0]
df_other_methods
df_other_methods=df_other_methods.rename(columns={'folds:':'dataset','0':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]


In [35]:
df_other_methods['method'].unique()

array(['SAST', 'ROCKET', 'HIVE-COTEv1', 'BOSS', 'RISE', 'TSF',
       'InceptionTime', 'TS-CHIEF', 'S-BOSS', 'STC', 'ResNet',
       'ProximityForest', 'WEASEL', 'Catch22', 'cBOSS'], dtype=object)

In [36]:
# Set directory where the csv files are located
#directory = os.getcwd()+'/ResultsRsast'
directory = os.getcwd()+'/results_accuracy_per_ds'
# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.startswith("df_overall_tunning") and filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df['rpoint']=df['rpoint'].astype(str)
        df['nb_per_class']=df['nb_per_class'].astype(str)

        
        # Append the dataframe to the list
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
df_result.head(3)


,Unnamed: 0,accuracy,time,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,dataset_name,classifier_name,rpoint,nb_per_class,method,len_method
0,0,0.930029,0.046570,0.006439,0.019473,0.017653,0.002193,Chinatown,ACF&PACF: n_random_points=1 nb_inst_per_class=...,1,(max instances per class)//2,Rsast,ACF&PACF
1,1,0.868805,0.040229,0.005562,0.003856,0.027773,0.002611,Chinatown,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,(lenthg ts)//2,1,Rsast,ACF&PACF
2,2,0.973761,0.098739,0.005058,0.018003,0.065576,0.009487,Chinatown,ACF&PACF: n_random_points= (lenthg ts)//2 nb_i...,(lenthg ts)//2,(max instances per class)//2,Rsast,ACF&PACF


In [37]:
#df_result=pd.read_csv("df_overall_tunning_SmoothSubspace3.csv")
df_result['dataset_name'].unique()


array(['Chinatown'], dtype=object)

In [38]:
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [39]:
# create a pivot table with the mean of score by dataset
len_method_vs_ds=pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['dataset_name'], aggfunc='mean')
len_method_vs_ds=np.transpose(len_method_vs_ds)#[['ACF','PACF']]
len_method_vs_ds=len_method_vs_ds.reset_index()
df_rocket=df_other_methods[df_other_methods["method"]=="STC"]
merged_df = len_method_vs_ds.merge(df_rocket,left_on='dataset_name', right_on='dataset',  how='left')
merged_df=merged_df.drop('method',axis=1)
merged_df

,dataset_name,ACF,ACF&PACF,Max ACF,Max PACF,None,PACF,dataset,score
0,Chinatown,0.955134,0.945416,0.93586,0.949789,0.932135,0.954001,Chinatown,0.96793


In [40]:
# create a pivot table with the mean of score by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10
len_method,,,
ACF,0.951166,0.958212,0.967930
ACF&PACF,0.947522,0.913508,0.968902
Max ACF,0.950680,0.838678,0.973761
Max PACF,0.963557,0.881438,0.963071
None,0.934645,0.912536,0.941691
PACF,0.951895,0.939747,0.976676


In [41]:
# create a pivot table with the mean of score by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10
len_method,,,
ACF,0.962585,0.902818,0.977648
ACF&PACF,0.955296,0.870748,0.980564
Max ACF,0.930272,0.927114,0.966958
Max PACF,0.942663,0.954325,0.973761
None,0.945821,0.832847,0.976676
PACF,0.966715,0.880466,0.976676


#### Time: Subsequence Lenght Method 

In [42]:
# create a pivot table with the mean of the time by dataset
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['dataset_name'], aggfunc='mean')

dataset_name,Chinatown
len_method,
ACF,0.052111
ACF&PACF,0.085551
Max ACF,0.048779
Max PACF,0.063174
None,0.042516
PACF,0.081628


In [43]:
# create a pivot table with the mean of the time by number of instances per class
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,(max instances per class)//2,1,10
len_method,,,
ACF,0.049393,0.035886,0.079207
ACF&PACF,0.085240,0.036663,0.135680
Max ACF,0.048090,0.026789,0.073525
Max PACF,0.066427,0.022664,0.090676
None,0.040710,0.024623,0.067634
PACF,0.075868,0.032684,0.153611


In [44]:
# create a pivot table with the mean of the time by number of random points
pd.pivot_table(df_result, values='time', index=['len_method'],columns=['rpoint'], aggfunc='mean')

rpoint,(lenthg ts)//2,1,10
len_method,,,
ACF,0.054932,0.030971,0.061968
ACF&PACF,0.090974,0.049464,0.099946
Max ACF,0.050884,0.033308,0.055829
Max PACF,0.059379,0.062614,0.078915
None,0.042330,0.047731,0.038048
PACF,0.090876,0.057359,0.068905


#### Variance: Subsequence Lenght Method 

In [45]:
# create a pivot table with the variance of the accuracy by number of instances per class
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,(max instances per class)//2,1,10
len_method,,,
ACF,0.000082,0.007840,0.000020
ACF&PACF,0.001175,0.010415,0.000011
Max ACF,0.004027,0.005074,0.000377
Max PACF,0.003331,0.000292,0.000008
None,0.000940,0.025366,0.000008
PACF,0.000410,0.003391,0.000161


In [46]:
# create a pivot table with the variance of the accuracy by number of random points
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['rpoint'], aggfunc='var')

rpoint,(lenthg ts)//2,1,10
len_method,,,
ACF,0.002355,0.000173,0.000025
ACF&PACF,0.004088,0.002808,0.000020
Max ACF,0.001336,0.004304,0.000059
Max PACF,0.000314,0.009574,0.000105
None,0.008496,0.001895,0.000059
PACF,0.002558,0.000462,0.000025


#### Another Results

In [47]:
df_result[['dataset_name','len_method','classifier_name']].describe()

,dataset_name,len_method,classifier_name
count,108,108,108
unique,1,6,30
top,Chinatown,ACF&PACF,None: n_random_points= (lenthg ts)//2 nb_inst_...
freq,108,18,6


In [48]:
pd.pivot_table(df_result, values='time', index=['dataset_name'], aggfunc='sum')/(60*60)

,time
dataset_name,
Chinatown,0.001869


In [49]:
df_result['time'].sum()/(60*60)

0.0018687965472539254

#### Generate CD Diagram Tunning Hyperparameter

In [50]:
#create dataset to generate cd diagram
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
#df_perf.classifier_name.unique()

In [51]:
from cd_function import *
#generate hyperparameter tuning cd diagrams
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF&PACF:')], labels=True, title="ACF&PACF comparison", name_file='images_cd_diagram/cd-diagram_ACF&PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^PACF:')], labels=True, title="PACF comparison", name_file='images_cd_diagram/cd-diagram_PACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^ACF:')], labels=True, title="ACF comparison", name_file='images_cd_diagram/cd-diagram_ACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^None:')], labels=True, title="None comparison", name_file='images_cd_diagram/cd-diagram_None.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max ACF:')], labels=True, title="Max ACF comparison", name_file='images_cd_diagram/cd-diagram_MaxACF.png')
draw_cd_diagram(df_perf[df_perf.classifier_name.str.contains(r'^Max PACF:')], labels=True, title="Max PACF comparison", name_file='images_cd_diagram/cd-diagram_MaxPACF.png')
draw_cd_diagram(df_perf, labels=True, title="overall Hyperparameter Tunning", name_file='images_cd_diagram/cd-diagram_overall_tunning.png')



the null hypothesis over the entire classifiers cannot be rejected
the null hypothesis over the entire classifiers cannot be rejected
the null hypothesis over the entire classifiers cannot be rejected
the null hypothesis over the entire classifiers cannot be rejected
the null hypothesis over the entire classifiers cannot be rejected
the null hypothesis over the entire classifiers cannot be rejected


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


ValueError: zero_method 'wilcox' and 'pratt' do not work if x - y is zero for all elements.

In [ ]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,0
Max PACF: n_random_points=30 nb_inst_per_class=1,127.916667
Max ACF: n_random_points=30 nb_inst_per_class=1,123.250000
Max ACF: n_random_points=1 nb_inst_per_class=1,121.875000
Max ACF: n_random_points=10 nb_inst_per_class=1,117.541667
Max PACF: n_random_points=1 nb_inst_per_class=30,116.666667
...,...
ACF&PACF: n_random_points=100 nb_inst_per_class=50,42.750000
ACF&PACF: n_random_points=100 nb_inst_per_class=30,42.375000
PACF: n_random_points=100 nb_inst_per_class=30,41.375000
ACF&PACF: n_random_points=10 nb_inst_per_class=100,41.125000


In [ ]:
#generate example cd diagrams
draw_cd_diagram(pd.read_csv('example_cd_diagram.csv'), labels=True, title="Example CD Diagram", name_file='cd-diagram_example.png')

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


#### Focus on most accurate lenght method

##### Overall Accuracy

In [ ]:
# highest accurate hyperparameters
average_ranks[average_ranks[0]==min_rank][0].index

Index(['ACF&PACF: n_random_points=50 nb_inst_per_class=100'], dtype='object')

In [ ]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100,30,50
rpoint,,,,,
1,0.716990,0.707127,0.813090,0.813616,0.809033
10,0.736965,0.814283,0.820416,0.818863,0.814046
100,0.800016,0.809959,0.815656,0.818897,0.818032
30,0.767948,0.805414,0.819020,0.816957,0.816813
50,0.807897,0.811014,0.825309,0.800429,0.821346


In [ ]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,1,10,100,30,50
rpoint,,,,,
1,0.066533,0.080303,0.033693,0.030073,0.040424
10,0.062014,0.033533,0.041195,0.039029,0.043808
100,0.028426,0.040994,0.046722,0.043260,0.044824
30,0.041493,0.049400,0.041841,0.038865,0.045356
50,0.024748,0.038161,0.040154,0.060122,0.044564


##### Overall time complexity

In [ ]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='mean')/60

,cweights_time,fsubsequence_time,tclassifier_time,tdataset_time,time
dataset_name,,,,,
BirdChicken,0.001775,1.845793,0.000515,3.106501,4.954678
Chinatown,0.000160,0.001823,0.000126,0.005264,0.007402
Coffee,0.001159,0.119599,0.000517,1.587549,1.708893
DistalPhalanxOutlineAgeGroup,0.000686,0.038889,0.010279,6.936574,6.986530
DistalPhalanxOutlineCorrect,0.000761,0.028729,0.012558,8.410944,8.453076
GunPoint,0.000717,0.042117,0.001210,1.425890,1.470008
ItalyPowerDemand,0.000144,0.003278,0.000300,0.030122,0.033879
ShapeletSim,0.002136,2.069743,0.000366,1.631211,3.703531
SmoothSubspace,0.000118,0.003916,0.000706,0.052012,0.056797


In [ ]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60

nb_per_class,1,10,100,30,50
rpoint,,,,,
1,0.059267,0.446878,0.508688,0.526805,0.518697
10,0.079947,0.725623,1.504154,1.075959,1.264428
100,0.355615,3.548282,11.194438,6.272142,8.778900
30,0.137414,1.335132,3.811814,2.261715,2.975149
50,0.211412,2.012466,6.171286,3.960022,4.480130


##### Calculate weights time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100,30,50
rpoint,,,,,
1,0.037933,0.035040,0.041002,0.037850,0.038401
10,0.040538,0.036245,0.047068,0.042641,0.043203
100,0.050934,0.043582,0.046869,0.049313,0.049783
30,0.044977,0.042565,0.048851,0.046539,0.050167
50,0.044484,0.045780,0.049981,0.045468,0.048524


##### Finding subsequences time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100,30,50
rpoint,,,,,
1,2.619535,24.856164,23.765545,27.900273,26.070745
10,2.788928,25.006459,24.043635,28.177670,26.608351
100,3.025260,28.780339,29.022799,26.399535,40.387128
30,2.542850,24.268097,30.311706,29.010061,31.668102
50,3.031036,30.201905,27.977514,32.043904,28.308357


##### Transform Dataset time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100,30,50
rpoint,,,,,
1,0.878649,1.860593,6.655589,3.620241,4.956047
10,1.918754,18.417328,66.034704,36.243088,49.119594
100,18.173916,183.906047,640.888910,348.941621,484.973417
30,5.594412,55.710952,196.670274,106.496284,146.490877
50,9.524848,90.367184,323.483648,178.356234,240.140503


##### Classifier time complexity

In [ ]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100,30,50
rpoint,,,,,
1,0.017345,0.059865,0.057892,0.048935,0.055273
10,0.047579,0.076098,0.120973,0.092173,0.092033
100,0.082500,0.160778,1.693620,0.927903,1.310779
30,0.061133,0.084119,1.670984,0.145335,0.294360
50,0.081386,0.128701,18.757385,27.149624,0.301612
